In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from collections import defaultdict
import re
import json
import os
import time
import arrow
import random

import numpy as np

import subprocess
import zipfile

from attraction import Attraction
from review import Review
from user import User

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
options.add_argument('--incognito')
options.add_argument('--start-maximized')
prefs = {"profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs",prefs)

# if headless:
#     options.add_argument('--headless')

In [26]:
driver = webdriver.Chrome('webdriver/chromedriver', options=options)

driver.get('https://www.tripadvisor.com.au/Attraction_Review-g255060-d257278-Reviews-Sydney_Opera_House-Sydney_New_South_Wales.html')

In [51]:
def tick_checkbox_item(item_name='Solo', max_attempts=3):
    
    times_tried = 0
    is_selected = False
    
    while (times_tried <= max_attempts) and (not is_selected):
        
        times_tried += 1
        
        try:
            WebDriverWait(driver, 10) \
                .until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
                                               f'div[data-tracker="{item_name}"]'))).click()
        except:
            
            print(f'failed to click on {item_name} filter..')

        try:
            
            WebDriverWait(driver, 10) \
                .until(EC.presence_of_element_located((By.CSS_SELECTOR, 
                                                   'input#filters_detail_checkbox_filterSegment__5[checked="checked"]')))
            is_selected = True
            
        except:
            
            print(f'failed to select {item_name}, re-trying...')
            
    if is_selected: 
        
        nrevs = 0
        
        try:
            
            WebDriverWait(driver, 10) \
                .until(EC.presence_of_element_located((By.CSS_SELECTOR, 
                                                   'input#filters_detail_language_filterLang_en[checked="checked"]')))
            
            nrevs = int(re.sub(r'[(,)]', '', driver.find_element_by_css_selector('label[for="filters_detail_language_filterLang_en"]>span.count').text))
            
            print(f'available reviews by {item_name} travellers: {nrevs:,}')
            
        except:
            
            langs = [_ for _ in driver.find_element_by_css_selector('div[data-prwidget-name="filters_detail_language"]').text.split('\n') if '(' in _]
            
            print(f'no English {item_name} reviews?')
            
            if langs:
                print(f'available review languages: {" | ".join(langs)}')
            else:
                print('there are no reviews in any language')          

In [52]:
tick_checkbox_item()

available reviews by Solo travellers: 1,717
